In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install pgmpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 5.2 MB/s 


In [ ]:
import numpy as np
import pandas as pd
from sklearn import svm, metrics
import matplotlib.pyplot as plt
#Load Data
msa_data = pd.read_csv('/content/drive/MyDrive/MLC Final Project/4_MSA_LQ.csv')
pop_data = pd.read_csv('/content/drive/MyDrive/MLC Final Project/1_PopChange_10_16.csv')
income_data = pd.read_csv('/content/drive/MyDrive/MLC Final Project/2_Income_Incr_10_16.csv')
emp_data = pd.read_csv('/content/drive/MyDrive/MLC Final Project/3_Emp_Incr_10_16.csv')

In [ ]:
#Merge MSA data and Population data
df_combine1 = msa_data.merge(pop_data[['msaID','PopChange']], left_on='MSA',right_on='msaID').drop('msaID',axis=1)
#Income
df_combine2 = df_combine1.merge(income_data[['msaID','IncomeChange']], left_on='MSA',right_on='msaID').drop('msaID',axis=1)
df_combine = df_combine2.merge(emp_data[['msaID','EmpChange']], left_on='MSA',right_on='msaID').drop('msaID',axis=1)
df_combine

,MSA,Agriculture,Mining_Oil_Gas,Utilities,Construction,Manufacturing,Wholesale_Trade,Retail_Trade,Transportation_Warehousing,Information,...,Administrative,Educational,Health_Care,Arts_Entertainment,Accommodation_Food,Other_Services,Public_Administration,PopChange,IncomeChange,EmpChange
0,10180,0.292018,3.878895,2.499900,1.364025,0.521449,0.914546,1.194708,0.950696,0.699227,...,0.414920,2.162690,1.405519,0.691371,1.113104,1.258570,0.921930,0.696193,0.532545,0.444016
1,10420,0.167061,0.181680,0.000000,0.909707,1.439877,1.333244,1.052671,0.927342,0.669126,...,0.577252,0.666918,1.158047,0.844240,0.919437,1.004739,0.370126,-0.031332,0.595657,0.660257
2,10500,5.062173,0.000000,0.000000,0.942033,0.973088,1.132075,1.416990,1.049779,0.651791,...,0.937060,0.176275,1.304347,0.410164,0.992618,1.017978,0.000000,-0.728060,0.609236,0.415536
3,10540,18.157445,0.296347,0.000000,1.259852,2.204625,0.951075,1.177175,2.081078,0.337859,...,0.570363,0.277039,0.969812,0.296457,0.866384,1.128558,1.669576,1.191507,1.119738,1.004935
4,10580,0.543236,0.012983,0.200071,0.971409,0.703094,0.858792,1.095503,0.765197,0.960970,...,0.489362,2.078170,1.243859,1.221169,0.932402,1.008777,1.022100,0.287495,0.674361,0.809862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,49420,22.348098,0.118864,0.731806,0.973424,1.467431,1.748183,1.198020,1.481026,0.495774,...,0.377055,0.722833,1.290262,1.516770,0.868819,0.872101,0.974948,0.592633,1.279254,0.618429
371,49620,0.063073,0.328722,3.401218,1.105633,2.013560,1.117206,1.057729,1.720350,0.448179,...,0.589948,0.680893,0.969834,1.053326,0.784367,1.324721,0.516259,0.453861,0.272015,0.653469
372,49660,0.100028,0.692022,1.160786,0.835405,1.844095,0.798913,1.198787,0.972419,0.525659,...,0.672242,0.624075,1.363910,0.847781,0.973651,1.088637,0.427168,-0.836413,0.889407,0.104659
373,49700,9.852723,1.075040,2.796943,1.004172,0.695962,1.396940,1.546210,0.884902,0.510613,...,0.530371,0.995936,1.285689,0.785467,1.097495,0.898688,1.974986,0.678833,1.000350,0.623180


# Variable bin = median, target bin = (-∞,0,1,+∞)

In [ ]:
df1 = df_combine
for column in df1.columns:
  if column != 'MSA' and column != 'PopChange' and column != 'IncomeChange' and column != 'EmpChange':
    df1[column] = df1[column]>=df1[column].median()
pop_change = []
income_change = []
emp_change = []
for i in range(len(df1)):
  if df1['PopChange'][i] < 0:
    pop_change.append('Decrease')
  elif ((df1['PopChange'][i] >= 0) & (df1['PopChange'][i] <= 1)):
    pop_change.append('Small Increase')
  elif (df1['PopChange'][i] > 1):
    pop_change.append('Obviously Increase')

for i in range(len(df1)):
  if df1['IncomeChange'][i] < 0:
    income_change.append('Decrease')
  elif ((df1['IncomeChange'][i] >= 0) & (df1['IncomeChange'][i] <= 1)):
    income_change.append('Small Increase')
  elif (df1['IncomeChange'][i] > 1):
    income_change.append('Obviously Increase')

for i in range(len(df1)):
  if df1['EmpChange'][i] < 0:
    emp_change.append('Decrease')
  elif ((df1['EmpChange'][i] >= 0) & (df1['EmpChange'][i] <= 1)):
    emp_change.append('Small Increase')
  elif (df1['EmpChange'][i] > 1):
    emp_change.append('Obviously Increase')


df1.PopChange = pd.Series(pop_change)
df1.IncomeChange = pd.Series(income_change)
df1.EmpChange = pd.Series(emp_change)
df1 = df1.rename(columns={'PopChange':'PopCategory','IncomeChange':'IncomeCategory','EmpChange':'EmpCategory'})
df1

,MSA,Agriculture,Mining_Oil_Gas,Utilities,Construction,Manufacturing,Wholesale_Trade,Retail_Trade,Transportation_Warehousing,Information,...,Administrative,Educational,Health_Care,Arts_Entertainment,Accommodation_Food,Other_Services,Public_Administration,PopCategory,IncomeCategory,EmpCategory
0,10180,False,True,True,True,False,True,True,True,True,...,False,True,True,False,True,True,True,Small Increase,Small Increase,Small Increase
1,10420,False,True,False,False,True,True,False,True,True,...,False,True,False,True,False,False,False,Decrease,Small Increase,Small Increase
2,10500,True,False,False,False,False,True,True,True,True,...,True,False,True,False,False,False,False,Decrease,Small Increase,Small Increase
3,10540,True,True,False,True,True,True,False,True,False,...,False,False,False,False,False,True,True,Obviously Increase,Obviously Increase,Obviously Increase
4,10580,True,False,False,False,False,False,False,False,True,...,False,True,True,True,False,False,True,Small Increase,Small Increase,Small Increase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,49420,True,False,False,False,True,True,True,True,False,...,False,True,True,True,False,False,True,Small Increase,Obviously Increase,Small Increase
371,49620,False,True,True,True,True,True,False,True,False,...,False,True,False,True,False,True,False,Small Increase,Small Increase,Small Increase
372,49660,False,True,True,False,True,False,True,True,False,...,True,False,True,True,False,True,False,Decrease,Small Increase,Small Increase
373,49700,True,True,True,True,False,True,True,False,False,...,False,True,True,False,True,False,True,Small Increase,Obviously Increase,Small Increase


In [ ]:
from sklearn.model_selection import train_test_split
df1_train, df1_test = train_test_split(df1,random_state=999,test_size=0.3)

In [ ]:
#PC
from pgmpy.estimators import PC
from pgmpy.estimators.CITests import chi_square

est = PC(df1_train)

skel, separating_sets = est.build_skeleton(significance_level=0.01)
print("Undirected edges: ", skel.edges())

pdag = est.skeleton_to_pdag(skel, separating_sets)
print("PDAG edges:       ", pdag.edges())

model = est.estimate()
print("DAG edges:        ", model.edges())

  0%|          | 0/5 [00:00<?, ?it/s]

Undirected edges:  [('Construction', 'PopCategory'), ('Manufacturing', 'Real_Estate'), ('Wholesale_Trade', 'Transportation_Warehousing'), ('Retail_Trade', 'Management'), ('Information', 'Professional_Services'), ('Professional_Services', 'Arts_Entertainment'), ('Accommodation_Food', 'PopCategory'), ('PopCategory', 'EmpCategory'), ('IncomeCategory', 'EmpCategory')]
PDAG edges:        [('IncomeCategory', 'EmpCategory'), ('Construction', 'PopCategory'), ('Accommodation_Food', 'PopCategory'), ('Professional_Services', 'Arts_Entertainment'), ('Professional_Services', 'Information'), ('Arts_Entertainment', 'Professional_Services'), ('Retail_Trade', 'Management'), ('Management', 'Retail_Trade'), ('Information', 'Professional_Services'), ('Real_Estate', 'Manufacturing'), ('Manufacturing', 'Real_Estate'), ('Transportation_Warehousing', 'Wholesale_Trade'), ('Wholesale_Trade', 'Transportation_Warehousing')]


  0%|          | 0/5 [00:00<?, ?it/s]

DAG edges:         [('IncomeCategory', 'EmpCategory'), ('Construction', 'PopCategory'), ('Accommodation_Food', 'PopCategory'), ('Professional_Services', 'Arts_Entertainment'), ('Management', 'Retail_Trade'), ('Information', 'Professional_Services'), ('Real_Estate', 'Manufacturing'), ('Wholesale_Trade', 'Transportation_Warehousing')]


In [ ]:
from pgmpy.models import BayesianModel
model1 = BayesianModel(model.edges())


/usr/local/lib/python3.7/dist-packages/pgmpy/models/BayesianModel.py:10: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  FutureWarning,


In [ ]:
from pgmpy.estimators import BayesianEstimator
model1.fit(df1_train, estimator=BayesianEstimator, prior_type='K2')
for cpd in model1.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

CPD of IncomeCategory:
+------------------------------------+-----------+
| IncomeCategory(Decrease)           | 0.0377358 |
+------------------------------------+-----------+
| IncomeCategory(Obviously Increase) | 0.373585  |
+------------------------------------+-----------+
| IncomeCategory(Small Increase)     | 0.588679  |
+------------------------------------+-----------+
CPD of EmpCategory:
+---------------------------------+-----+--------------------------------+
| IncomeCategory                  | ... | IncomeCategory(Small Increase) |
+---------------------------------+-----+--------------------------------+
| EmpCategory(Decrease)           | ... | 0.06962025316455696            |
+---------------------------------+-----+--------------------------------+
| EmpCategory(Obviously Increase) | ... | 0.17088607594936708            |
+---------------------------------+-----+--------------------------------+
| EmpCategory(Small Increase)     | ... | 0.759493670886076              |


In [ ]:
node_list = []
for i in list(model.edges()):
  if i[0] not in ['PopCategory','IncomeCategory','EmpCategory']:
    node_list.append(i[0])
  if i[1] not in ['PopCategory','IncomeCategory','EmpCategory']:
    node_list.append(i[1])
node_list = list(set(node_list))
node_list

['Real_Estate',
 'Arts_Entertainment',
 'Retail_Trade',
 'Manufacturing',
 'Accommodation_Food',
 'Transportation_Warehousing',
 'Management',
 'Construction',
 'Information',
 'Professional_Services',
 'Wholesale_Trade']

In [ ]:
model1.predict(df1_test.loc[:,node_list])

  0%|          | 0/92 [00:00<?, ?it/s]

,IncomeCategory,PopCategory,EmpCategory
0,Small Increase,Obviously Increase,Small Increase
1,Small Increase,Decrease,Small Increase
2,Small Increase,Decrease,Small Increase
3,Small Increase,Obviously Increase,Small Increase
4,Small Increase,Small Increase,Small Increase
...,...,...,...
108,Small Increase,Small Increase,Small Increase
109,Small Increase,Decrease,Small Increase
110,Small Increase,Obviously Increase,Small Increase
111,Small Increase,Small Increase,Small Increase


In [ ]:
predicted_train = model1.predict(df1_train.loc[:,node_list])
print("In sample for IncomeCategory:",(df1_train.loc[:,'IncomeCategory'].reset_index(drop=True)==predicted_train['IncomeCategory']).mean())
predicted_test = model1.predict(df1_test.loc[:,node_list])
print("Out of sample for IncomeCategory:",(df1_test.loc[:,'IncomeCategory'].reset_index(drop=True)==predicted_test['IncomeCategory']).mean())

  0%|          | 0/193 [00:00<?, ?it/s]

In sample for IncomeCategory: 0.5916030534351145


  0%|          | 0/92 [00:00<?, ?it/s]

Out of sample for IncomeCategory: 0.6460176991150443


In [ ]:
predicted_train = model1.predict(df1_train.loc[:,node_list])
print("In sample for EmpCategory:",(df1_train.loc[:,'EmpCategory'].reset_index(drop=True)==predicted_train['EmpCategory']).mean())
predicted_test = model1.predict(df1_test.loc[:,node_list])
print("Out of sample for EmpCategory:",(df1_test.loc[:,'EmpCategory'].reset_index(drop=True)==predicted_test['EmpCategory']).mean())

  0%|          | 0/193 [00:00<?, ?it/s]

In sample for EmpCategory: 0.6793893129770993


  0%|          | 0/92 [00:00<?, ?it/s]

Out of sample for EmpCategory: 0.6637168141592921


In [ ]:
predicted_train = model1.predict(df1_train.loc[:,node_list])
print("In sample for PopCategory:",(df1_train.loc[:,'PopCategory'].reset_index(drop=True)==predicted_train['PopCategory']).mean())
predicted_test = model1.predict(df1_test.loc[:,node_list])
print("Out of sample for PopCategory:",(df1_test.loc[:,'PopCategory'].reset_index(drop=True)==predicted_test['PopCategory']).mean())

  0%|          | 0/193 [00:00<?, ?it/s]

In sample for PopCategory: 0.549618320610687


  0%|          | 0/92 [00:00<?, ?it/s]

Out of sample for PopCategory: 0.5575221238938053
